## Setup

In [171]:
import sys, os, json, csv

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt  
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS 

sns.set_theme(font_scale=1.5)
%matplotlib inline
%load_ext autoreload
%autoreload 2

pd.options.display.max_rows = 90
pd.options.display.max_columns = 90
pd.set_option('display.max_colwidth', 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Data

In [172]:
# df = pd.read_csv('data/AB_NYC_2019.csv')  
# df = pd.read_csv('data/AB_US_2020.csv')  
df = pd.read_csv('data/airbnb-listings.csv', sep=';', low_memory=False)

# Overview

In this project, data for Airbnb listings is explored in an effort to glean insights relating to rental properties. An exhaustive hypotheses testing and modeling process will be performed in order to offer guidance to property owners renting through Airbnb or similar platforms.

## About the Datasets

The datasets used in this project were retrieved from various sources, listed below.  

  [Airbnb — Listings](https://public.opendatasoft.com/explore/dataset/airbnb-listings/information/?disjunctive.host_verifications&disjunctive.amenities&disjunctive.features&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQ09VTlQiLCJ5QXhpcyI6Imhvc3RfbGlzdGluZ3NfY291bnQiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiJyYW5nZS1jdXN0b20ifV0sInhBeGlzIjoiY2l0eSIsIm1heHBvaW50cyI6IiIsInRpbWVzY2FsZSI6IiIsInNvcnQiOiIiLCJzZXJpZXNCcmVha2Rvd24iOiJyb29tX3R5cGUiLCJjb25maWciOnsiZGF0YXNldCI6ImFpcmJuYi1saXN0aW5ncyIsIm9wdGlvbnMiOnsiZGlzanVuY3RpdmUuaG9zdF92ZXJpZmljYXRpb25zIjp0cnVlLCJkaXNqdW5jdGl2ZS5hbWVuaXRpZXMiOnRydWUsImRpc2p1bmN0aXZlLmZlYXR1cmVzIjp0cnVlfX19XSwidGltZXNjYWxlIjoiIiwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D)  

  [Kaggle — New York City Airbnb Open Data](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data)  
  Airbnb listings and metrics in NYC, NY, USA (2019)  
  - Since 2008, guests and hosts have used Airbnb to expand on traveling possibilities and present more unique, personalized way of experiencing the world. This dataset describes the listing activity and metrics in NYC, NY for 2019.  
  - This data file includes all needed information to find out more about hosts, geographical availability, necessary metrics to make predictions and draw conclusions.  
  
[Kaggle — U.S. Airbnb Open Data](https://www.kaggle.com/kritikseth/us-airbnb-open-data)  
  Airbnb listings and metrics 

# Cleaning

In [169]:
df.columns = df.columns.str.lower() # set column names to lowercase

df = df.drop(labels=37367, axis=0, inplace=True) # drop dirty record

df = df.price.dropna(axis=0, inplace=True) # drop records with missing price

In [170]:
us_df = df[df.country == 'United States'] # DataFrame of only listings located in USA
ma_df = us_df[us_df.state == 'MA']

AttributeError: 'NoneType' object has no attribute 'country'

# EDA

### Initial

In [130]:
df.columns

Index(['id', 'listing url', 'scrape id', 'last scraped', 'name', 'summary',
       'space', 'description', 'experiences offered', 'neighborhood overview',
       'notes', 'transit', 'access', 'interaction', 'house rules',
       'thumbnail url', 'medium url', 'picture url', 'xl picture url',
       'host id', 'host url', 'host name', 'host since', 'host location',
       'host about', 'host response time', 'host response rate',
       'host acceptance rate', 'host thumbnail url', 'host picture url',
       'host neighbourhood', 'host listings count',
       'host total listings count', 'host verifications', 'street',
       'neighbourhood', 'neighbourhood cleansed',
       'neighbourhood group cleansed', 'city', 'state', 'zipcode', 'market',
       'smart location', 'country code', 'country', 'latitude', 'longitude',
       'property type', 'room type', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'bed type', 'amenities', 'square feet', 'price', 'weekly price',
       'month

In [115]:
df.head(3)

,ID,Listing Url,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,Notes,Transit,Access,Interaction,House Rules,Thumbnail Url,Medium Url,Picture Url,XL Picture Url,Host ID,Host URL,Host Name,Host Since,Host Location,Host About,Host Response Time,Host Response Rate,Host Acceptance Rate,Host Thumbnail Url,Host Picture Url,Host Neighbourhood,Host Listings Count,Host Total Listings Count,Host Verifications,Street,Neighbourhood,Neighbourhood Cleansed,Neighbourhood Group Cleansed,City,State,Zipcode,Market,Smart Location,Country Code,Country,Latitude,Longitude,Property Type,Room Type,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Square Feet,Price,Weekly Price,Monthly Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Minimum Nights,Maximum Nights,Calendar Updated,Has Availability,Availability 30,Availability 60,Availability 90,Availability 365,Calendar last Scraped,Number of Reviews,First Review,Last Review,Review Scores Rating,Review Scores Accuracy,Review Scores Cleanliness,Review Scores Checkin,Review Scores Communication,Review Scores Location,Review Scores Value,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Reviews per Month,Geolocation,Features
0,17619081,https://www.airbnb.com/rooms/17619081,20170615002708,2017-06-15,Nice smallroom i central old Frederiksberg,Min bolig lægger tæt på Bus og Metro. Frederik...,NaN,Min bolig lægger tæt på Bus og Metro. Frederik...,none,NaN,NaN,The appartement is 800 metre to Frederiksberg ...,You have a small room and and can use the kitc...,NaN,- ingen sko indendørs. Ingen støj,https://a0.muscache.com/im/pictures/d2da4bbd-4...,https://a0.muscache.com/im/pictures/d2da4bbd-4...,https://public.opendatasoft.com/api/v2/catalog...,https://a0.muscache.com/im/pictures/d2da4bbd-4...,39150424,https://www.airbnb.com/users/show/39150424,Birgitte,2015-07-21,"Frederiksberg, Capital Region of Denmark, Denmark",Trekking is in my heart. Walking in the mounta...,within an hour,100.0,NaN,https://a0.muscache.com/im/pictures/f5c5b44f-f...,https://a0.muscache.com/im/pictures/f5c5b44f-f...,Frederiksberg,2.0,2.0,"email,phone,facebook,linkedin,reviews,jumio,go...","Frederiksberg, Frederiksberg, 1855, Denmark",Frederiksberg,Frederiksberg,NaN,Frederiksberg,NaN,1855,Copenhagen,"Frederiksberg, Denmark",DK,Denmark,55.675979,12.532960,Apartment,Private room,1.0,1.0,1.0,1.0,Real Bed,"Wireless Internet,Kitchen,Smoke detector,Essen...",NaN,298.0,NaN,NaN,NaN,NaN,1.0,0.0,2.0,5.0,today,NaN,6.0,6.0,7.0,91.0,2017-06-15,2.0,2017-03-17,2017-05-03,90.0,10.0,10.0,10.0,10.0,8.0,10.0,NaN,NaN,flexible,2.0,0.66,"55.6759793551,12.5329597787","Host Is Superhost,Host Has Profile Pic,Host Id..."
1,13356620,https://www.airbnb.com/rooms/13356620,20170615002708,2017-06-15,Spacious 95 m2 3 room apt. with great location,Spacious and sunny apt. located close to the b...,The apt. is located on 3rd floor and faces a t...,Spacious and sunny apt. located close to the b...,none,NaN,NaN,"Getting here from CPH airport is easy, take th...",Entire flat.,NaN,NaN,NaN,NaN,https://public.opendatasoft.com/api/v2/catalog...,NaN,75748444,https://www.airbnb.com/users/show/75748444,Torsten,2016-06-04,"Frederiksberg, Capital Region of Denmark, Denmark",I live with my girlfriend and our baby daughte...,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/9e72aa5b-9...,https://a0.muscache.com/im/pictures/9e72aa5b-9...,Frederiksberg,1.0,1.0,"email,phone,linkedin,reviews,jumio,government_id","Frederiksberg, Frederiksberg, 1855, Denmark",Frederiksberg,Frederiksberg,NaN,Frederiksberg,NaN,1855,NaN,"Frederiksberg, Denmark",DK,Denmark,55.675909,12.534326,Apartment,Entire home/apt,3.0,1.0,2.0,3.0,Real Bed,"TV,Wireless Internet,Kitchen,Heating,Family/ki...",NaN,NaN,NaN,NaN,993.0,364.0,2.0,132.0,2.0,28.0,7 months ago,NaN,0.0,0.0,0.0,0.0,2017-06-15,5.0,2016-06-15,2016-09-11,96.0,10.0,10.0,10.0,10.0,10.0,9.0,NaN,NaN,strict,1.0,0.41,"55.6759094982,12.5343258647","Host Has Profile Pic,Host I

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494954 entries, 0 to 494953
Data columns (total 89 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              494954 non-null  object 
 1   Listing Url                     494954 non-null  object 
 2   Scrape ID                       494954 non-null  object 
 3   Last Scraped                    494953 non-null  object 
 4   Name                            494536 non-null  object 
 5   Summary                         478753 non-null  object 
 6   Space                           347019 non-null  object 
 7   Description                     494794 non-null  object 
 8   Experiences Offered             494954 non-null  object 
 9   Neighborhood Overview           291604 non-null  object 
 10  Notes                           197643 non-null  object 
 11  Transit                         303786 non-null  object 
 12  Access          

In [59]:
df_nulls = df.isna().sum()

In [110]:
df = df.dropna(axis=0, inplace=True)

In [111]:
df

## state

In [151]:
len(us_df[us_df.state == 'MA'])

3585

## country

In [145]:
df.country.unique()

array(['Denmark', 'United States', 'United Kingdom', 'Belgium',
       'Hong Kong', 'China', 'Spain', 'Ireland', 'Australia', 'Canada',
       'Switzerland', 'France', 'Cuba', 'Germany', 'Netherlands',
       'Greece', 'Italy', 'Vatican City', 'Austria', 'Vanuatu', 'Mexico',
       nan, 'Uruguay'], dtype=object)

In [146]:
len(df[df.country == 'United States'])

134545

In [148]:
us_df

,id,listing url,scrape id,last scraped,name,summary,space,description,experiences offered,neighborhood overview,notes,transit,access,interaction,house rules,thumbnail url,medium url,picture url,xl picture url,host id,host url,host name,host since,host location,host about,host response time,host response rate,host acceptance rate,host thumbnail url,host picture url,host neighbourhood,host listings count,host total listings count,host verifications,street,neighbourhood,neighbourhood cleansed,neighbourhood group cleansed,city,state,zipcode,market,smart location,country code,country,latitude,longitude,property type,room type,accommodates,bathrooms,bedrooms,beds,bed type,amenities,square feet,price,weekly price,monthly price,security deposit,cleaning fee,guests included,extra people,minimum nights,maximum nights,calendar updated,has availability,availability 30,availability 60,availability 90,availability 365,calendar last scraped,number of reviews,first review,last review,review scores rating,review scores accuracy,review scores cleanliness,review scores checkin,review scores communication,review scores location,review scores value,license,jurisdiction names,cancellation policy,calculated host listings count,reviews per month,geolocation,features
241,16163330,https://www.airbnb.com/rooms/16163330,20170510102046,2017-05-10,Quiet Space to sleep & rest,"Hello. Small bedroom with twin bed, chair, ni...",NaN,"Hello. Small bedroom with twin bed, chair, ni...",none,NaN,NaN,NaN,NaN,Pretty hands off but available for questions a...,NaN,https://a0.muscache.com/im/pictures/745571e6-2...,https://a0.muscache.com/im/pictures/745571e6-2...,https://public.opendatasoft.com/api/v2/catalog...,https://a0.muscache.com/im/pictures/745571e6-2...,3836635,https://www.airbnb.com/users/show/3836635,Caitlin,2012-10-11,"Chicago, Illinois, United States","I am a happy, relaxed individual that enjoys c...",within an hour,100.0,NaN,https://a0.muscache.com/im/users/3836635/profi...,https://a0.muscache.com/im/users/3836635/profi...,Humboldt Park,2.0,2.0,"email,phone,reviews","Humboldt Park, Chicago, IL 60647, United States",Humboldt Park,Humboldt Park,NaN,Chicago,IL,60647,Chicago,"Chicago, IL",US,United States,41.911282,-87.702463,Apartment,Private room,1.0,1.0,1.0,1.0,Real Bed,"Wireless Internet,Air conditioning,Kitchen,Hea...",NaN,20.0,NaN,NaN,NaN,7.0,1.0,0.0,3.0,1125.0,2 weeks ago,NaN,2.0,2.0,2.0,2.0,2017-05-10,15.0,2017-01-02,2017-04-29,100.0,10.0,10.0,10.0,10.0,9.0,10.0,NaN,"Illinois State, Cook County, IL, CHICAGO",strict,2.0,3.49,"41.9112817535,-87.7024631364","Host Has Profile Pic,Is Location Exact,Require..."
242,6846209,https://www.airbnb.com/rooms/6846209,20170510102046,2017-05-10,Rogue Philanthropy Manor,Our Humboldt Park home features 1.5 bathrooms ...,Old 120-year-old farm house appeal. Not a typ...,Our Humboldt Park home features 1.5 bathrooms ...,none,Humboldt Park is an emerging Chicago neighborh...,"My husband, Dave, is a local Chicago chef and ...",We are right by the North Ave bus that will ta...,"Aside from the bedroom, guests are welcome to ...",We'd love to help you navigate the city and gi...,"Smoking outside, please. We have an old Choco...",https://a0.muscache.com/im/pictures/90605110/6...,https://a0.muscache.com/im/pictures/90605110/6...,https://public.opendatasoft.com/api/v2/catalog...,https://a0.muscache.com/im/pictures/90605110/6...,35874075,https://www.airbnb.com/users/show/35874075,Denise,2015-06-15,"Chicago, Illinois, United States",I'm a Chicago transplant via the Quad Cities a...,within a day,100.0,NaN,https://a0.muscache.com/im/users/35874075/prof...,https://a0.muscache.com/im/users/35874075/prof...,Humboldt Park,1.0,1.0,"email,phone,reviews,kba","Humboldt Park, Chicago, IL 60651, United States",Humboldt Park,Humboldt Park,NaN,Chicago,IL,60651,Chicago,"Chicago, IL",US,United States,41.906670,-87.714291,House,Private room,2.0,1.5,1.0,1.0,Real Bed,"Internet,Wireless Internet,Air conditioning,Ki...",NaN,60.0,400.0,1200.0,NaN,10.0,2

# EDA of Massachusetts Listings

In [154]:
ma_df.columns

Index(['id', 'listing url', 'scrape id', 'last scraped', 'name', 'summary',
       'space', 'description', 'experiences offered', 'neighborhood overview',
       'notes', 'transit', 'access', 'interaction', 'house rules',
       'thumbnail url', 'medium url', 'picture url', 'xl picture url',
       'host id', 'host url', 'host name', 'host since', 'host location',
       'host about', 'host response time', 'host response rate',
       'host acceptance rate', 'host thumbnail url', 'host picture url',
       'host neighbourhood', 'host listings count',
       'host total listings count', 'host verifications', 'street',
       'neighbourhood', 'neighbourhood cleansed',
       'neighbourhood group cleansed', 'city', 'state', 'zipcode', 'market',
       'smart location', 'country code', 'country', 'latitude', 'longitude',
       'property type', 'room type', 'accommodates', 'bathrooms', 'bedrooms',
       'beds', 'bed type', 'amenities', 'square feet', 'price', 'weekly price',
       'month

In [156]:
ma_df['property type'].value_counts()

Apartment          2612
House               562
Condominium         231
Townhouse            54
Bed & Breakfast      41
Loft                 39
Other                17
Boat                 12
Villa                 6
Entire Floor          4
Dorm                  2
Camper/RV             1
Guesthouse            1
Name: property type, dtype: int64

In [157]:
ma_df['room type'].value_counts()

Entire home/apt    2127
Private room       1378
Shared room          80
Name: room type, dtype: int64

In [167]:
ma_df.price.isna().sum()

12

<br><br><br><br>

# Bibliography

All Datasets used have been made available under Creative Commons CC0 1.0 Universal (CC0 1.0) "Public Domain Dedication"
  
  
  
  [Airbnb — Listings](https://public.opendatasoft.com/explore/dataset/airbnb-listings/information/?disjunctive.host_verifications&disjunctive.amenities&disjunctive.features&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQ09VTlQiLCJ5QXhpcyI6Imhvc3RfbGlzdGluZ3NfY291bnQiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiJyYW5nZS1jdXN0b20ifV0sInhBeGlzIjoiY2l0eSIsIm1heHBvaW50cyI6IiIsInRpbWVzY2FsZSI6IiIsInNvcnQiOiIiLCJzZXJpZXNCcmVha2Rvd24iOiJyb29tX3R5cGUiLCJjb25maWciOnsiZGF0YXNldCI6ImFpcmJuYi1saXN0aW5ncyIsIm9wdGlvbnMiOnsiZGlzanVuY3RpdmUuaG9zdF92ZXJpZmljYXRpb25zIjp0cnVlLCJkaXNqdW5jdGl2ZS5hbWVuaXRpZXMiOnRydWUsImRpc2p1bmN0aXZlLmZlYXR1cmVzIjp0cnVlfX19XSwidGltZXNjYWxlIjoiIiwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D)  
  - Dataset available under Creative Commons CC0: Public Domain License, 2020  
  - [Available from public.opendatasoft.com](https://public.opendatasoft.com/explore/dataset/airbnb-listings/information/?disjunctive.host_verifications&disjunctive.amenities&disjunctive.features&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQ09VTlQiLCJ5QXhpcyI6Imhvc3RfbGlzdGluZ3NfY291bnQiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiJyYW5nZS1jdXN0b20ifV0sInhBeGlzIjoiY2l0eSIsIm1heHBvaW50cyI6IiIsInRpbWVzY2FsZSI6IiIsInNvcnQiOiIiLCJzZXJpZXNCcmVha2Rvd24iOiJyb29tX3R5cGUiLCJjb25maWciOnsiZGF0YXNldCI6ImFpcmJuYi1saXN0aW5ncyIsIm9wdGlvbnMiOnsiZGlzanVuY3RpdmUuaG9zdF92ZXJpZmljYXRpb25zIjp0cnVlLCJkaXNqdW5jdGl2ZS5hbWVuaXRpZXMiOnRydWUsImRpc2p1bmN0aXZlLmZlYXR1cmVzIjp0cnVlfX19XSwidGltZXNjYWxlIjoiIiwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D)  
  - Originally source from [Inside Airbnb](
http://insideairbnb.com/)
  
[New York City Airbnb Open Data](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data)  
  - Dataset available under Creative Commons CC0: Public Domain License, 2019  
  - Uploaded to Kaggle by user [Dgomonov](https://www.kaggle.com/dgomonov)
  - Originally sourced from [Inside Airbnb](
http://insideairbnb.com/)
  
[U.S. Airbnb Open Data](https://www.kaggle.com/kritikseth/us-airbnb-open-data)  
  - Dataset available under Creative Commons CC0: Public Domain License, 2020  
  - Uploaded to Kaggle by user [Kritik Seth](https://www.kaggle.com/kritikseth)  
  - Originally sourced from [Inside Airbnb](
http://insideairbnb.com/)
    
  

<br><br><br><br>